In [ ]:
####################### I M P O R T S ###############################

import os
import glob
import numpy as np
from os.path import dirname, basename
from openmm.app import *
from openmm import *
from openmm.unit import kelvin, nanometer, picoseconds
from sys import stdout
from herobm.utils.minimisation import minimise

#####################################################################

# Minimise Structure #

If there are problems, run 'openmm-setup'

In [ ]:
####################### C O N F I G #################################

PDB_FOLDER = "backmapped/Vince/martini3/" # "comparison/herobm/" # "backmapped/CG-all/protein-whole/"
PDB_REGEX = "**/backmapped_fixed_*.pdb" # "**/4_final_de_novo*.pdb"
TOLERANCE = 50 # kilojoules_per_mole/nanometer
FOLDER_NAME = "minimised"
ADD_MEMBRANE = False
SOLVATE = False
RESTRAIN_ATOMS = ['N', 'CA', 'C', 'O']

#####################################################################



for pdb_in_filename in [log for log in glob.glob(os.path.join(PDB_FOLDER, PDB_REGEX), recursive=True) if not basename(dirname(log)) == 'minimized']:
        pdb_folder = dirname(pdb_in_filename)
        minimised_pdb_folder = os.path.join(pdb_folder, FOLDER_NAME)
        os.makedirs(minimised_pdb_folder, exist_ok=True)
        pdb_out_filename = os.path.join(
                minimised_pdb_folder,
                basename(pdb_in_filename),
        )
        if os.path.isfile(pdb_out_filename):
                continue
        minimise(pdb_in_filename, pdb_out_filename)

In [ ]:
from herobm.utils.pdbFixer import fixPDB, joinPDBs
joinPDBs(os.path.join(PDB_FOLDER, 'martini3_bbcommon/minimised'), 'backmapped')

# Run Simulation #

In [ ]:
for filename in glob.glob(os.path.join(PDB_FOLDER, f"backmapped_*.pdb")):
        pdb = PDBFile(pdb_in_filename)
        forcefield = ForceField('amber14-all.xml')
        system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
                nonbondedCutoff=1*nanometer, constraints=HBonds)
        while system.getNumForces() > 0:
                system.removeForce(0)
        while system.getNumConstraints() > 0:
                system.removeConstraint(0)
        
        assert system.getNumConstraints() == 0
        assert system.getNumForces() == 0
        integrator = VerletIntegrator(0.001*picoseconds)
        simulation = Simulation(pdb.topology, system, integrator)
        simulation.context.setPositions(pdb.positions)

        simulation_pdb_folder = os.path.join(PDB_FOLDER, 'simulation')
        os.makedirs(simulation_pdb_folder, exist_ok=True)

        simulation.reporters.append(PDBReporter(os.path.join(simulation_pdb_folder, basename(filename)), 10000))
        simulation.reporters.append(
        StateDataReporter(
                stdout, 10000, step=True,potentialEnergy=True, temperature=True)
        )

        os.makedirs(os.path.join(simulation_pdb_folder), exist_ok=True)

        simulation.minimiseEnergy(tolerance=TOLERANCE*unit.kilojoules_per_mole/nanometer, maxIterations=10000)
        simulation.step(500000)
        break